In [1]:
import os
import pandas as pd
import pathlib
import re
from glob import glob
import xml.etree.ElementTree as ET
import spacy 
from spacy import displacy

In [ ]:
#!pip install git+git://github.com/petermr/pygetpapers
QUERY = "ethics statement frontiers"
HITS = 100
OUTPUT = 'ethics_statement_frontiers_100'
!pygetpapers -q "$QUERY" -k $HITS -o $OUTPUT -x


In [ ]:
#!git clone https://github.com/petermr/ami3.git
#!cd ami3
#!mvn install -Dmaven.test.skip=true
!ami -p $OUTPUT section

In [ ]:
HOME= 'C:/Users/shweata/dictionary/ethics_statement_project'
ethics_statements = glob(os.path.join(HOME, OUTPUT, 'PMC*', 'sections','*', '[0-9]_ethic*', '[1_9]_p.xml'))
ethics_statements
#len(ethics_statements)

In [ ]:

#path = os.path.join(HOME, 'e_cancer_clinical_trial_50', 'PMC*', 'sections','*', '*', '[1_9]_p.xml')
#normalized_path = os.path.normpath(path)
search_term = "ethic"

files_to_check = []

# looping through all the filenames returned
# returns only the fulltext.xml since we don't look in the sub-directories
# set recursive = False - doesn't look in sub-directories
for filename in glob(os.path.join(HOME, 'e_cancer_clinical_trial_50', 'PMC*', 'sections','*', '*', '*.xml'), recursive=False):
    # adding error handling just in case!
    try:
        with open(filename) as f:
            # read the file as a string
            contents = f.read()
            # if the search term is found append to the list of files
            if(search_term in contents):
                files_to_check.append(filename)
    except:
        pass
files_to_check
#len(ethics_statements)

In [ ]:
file1 = open("ethics_statements_frontiers_100.txt","w+", encoding='utf-8')
for file in ethics_statements:
    tree = ET.parse(file)
    root = tree.getroot()
    for para in root.iter('p'):
        #file1.write('para.text')
        print (para.text, file = file1)

text = pathlib.Path("ethics_statements_frontiers_100.txt").read_text(encoding='utf-8')

In [ ]:
nlp = spacy.load("en_core_web_sm")


#!python -m pip install icecream
#from icecream import ic
doc = nlp(text)


entities = []
labels = []
position_start = []
position_end = []

for ent in doc.ents:
    entities.append(ent)
    labels.append(ent.label_)
    position_start.append(ent.start_char)
    position_end.append(ent.end_char)
    
df = pd.DataFrame({'Entities':entities,'Labels':labels,'Position_Start':position_start, 'Position_End':position_end})
#pd.set_option("display.max_rows", None, "display.max_columns", None)
df.to_csv('entity_recognition_spacy_ethics_statement_frontiers_100.csv', encoding='utf-8')
#ic(entities, labels)


In [ ]:
pattern = re.compile(r'approved\sby\sthe\sEthics\sCommittee\sof\s[a-zA-Z]*.')
matches=pattern.finditer(text)
for match in matches:
    print(match)

In [ ]:
# Trials/Experiments that didn't require ethics committee
pattern = re.compile(r'not\srequired\s[a-zA-Z]*.')
matches=pattern.findall(text)
for match in matches:
    print(match)
len(matches)

In [3]:
#!pip install sparqlwrapper

import sys
from SPARQLWrapper import SPARQLWrapper, JSON, XML
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11")



In [4]:
endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT DISTINCT ?org ?orgLabel ?acronym WHERE {
  ?org wdt:P3500 ?ringgold .
  ?org wdt:P1813 ?acronym .
  ?person wdt:P108 ?org .
  ?paper wdt:P50 ?person .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    
    sparql.setReturnFormat(XML)
    return sparql.query().convert()

results_xml = get_results(endpoint_url, query)

for result in results_xml["results"]["bindings"]:
    print(result)


TypeError: 'Document' object is not subscriptable

In [24]:
import sys
from SPARQLWrapper import SPARQLWrapper, XML

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?ethics_committee ?ethics_committeeLabel WHERE {
  ?ethics_committee wdt:P31 wd:Q59057226 .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
  """


def get_results_xml(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(XML)
    return sparql.query().convert().toxml()

results = get_results_xml(endpoint_url, query)
file2 = open("ethics_committee_sparql_test.xml","w+", encoding='utf-8')
print(results, file = file2)

In [6]:
import sys
from SPARQLWrapper import SPARQLWrapper, XML

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?ethics_committee ?ethics_committeeLabel WHERE {
  ?ethics_committee wdt:P31 wd:Q59057226 .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""


def get_results_xml(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(XML)
    return sparql.query().convert().toxml()

results = get_results_xml(endpoint_url, query)
file2 = open("ethics_committee.xml","w+", encoding='utf-8')
print (results, file = file2)

In [19]:
import os
os.system('amidict -vv --dictionary ethics_committee --directory plant_genus_dictionary --input ethics_committee_sparql.xml create --informat wikisparqlxml --sparqlmap wikidataURL=ethics_committee,term=ethics_committeeLabel,wikidataURL=ethics_committee --transformName wikidataID=EXTRACT(wikidataURL,.*/(.*))')

1